In [12]:
import torch
import numpy as np
torch.set_printoptions(edgeitems=2, linewidth=75)
t_c = [0.5,  14.0, 15.0, 28.0, 11.0,  8.0,  3.0, -4.0,  6.0, 13.0, 21.0] # 섭씨온도 
t_u = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4] # 모르는온도
t_c = torch.tensor(t_c)
t_u = torch.tensor(t_u)

In [18]:
def model(t_u, w, b):
    return w * t_u + b

def loss_fn(t_p,t_c):
    squared_diffs = (t_p-t_c)**2
    return squared_diffs.mean()

params = torch.tensor([1.0,0.0],requires_grad=True) # 자동미분 옵션(처음에는 NONE값이었다가 기울기를 누적시킨다)
loss = loss_fn(model(t_u,*params),t_c)
loss.backward()
params.grad #


tensor([4517.2969,   82.6000])

In [31]:
def training_loop(n_epochs, learning_rate, params, t_u, t_c):
    for epoch in range(1, n_epochs + 1):
        if params.grad is not None:  # 만약 초기화가 되지 않은 상태라면 초기화를 시켜준다
            params.grad.zero_()
        
        t_p = model(t_u, *params) 
        loss = loss_fn(t_p, t_c)
        loss.backward()
        
        with torch.no_grad(): 
            params -= learning_rate * params.grad
            # params = params - learning_rate * params.grad # 이렇게 바꾸면 ouplace연산이 되는데 pytorch는 자동미분을 위해 inplace연산을 해서 
        if epoch % 500 == 0:
            print('Epoch %d, Loss %f' % (epoch, float(loss)))
            
    return params

In [32]:
t_un = 0.1*t_u
training_loop(
    n_epochs = 5000, 
    learning_rate = 1e-2, 
    params = torch.tensor([1.0, 0.0], requires_grad=True), # <1> 
    t_u = t_un, # <2> 
    t_c = t_c)

Epoch 500, Loss 7.860115
Epoch 1000, Loss 3.828538
Epoch 1500, Loss 3.092191
Epoch 2000, Loss 2.957698
Epoch 2500, Loss 2.933134
Epoch 3000, Loss 2.928648
Epoch 3500, Loss 2.927830
Epoch 4000, Loss 2.927679
Epoch 4500, Loss 2.927652
Epoch 5000, Loss 2.927647


tensor([  5.3671, -17.3012], requires_grad=True)